In [1]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Menyembunyikan pesan INFO/WARNING
import tensorflow as tf
tf.get_logger().setLevel('ERROR')  # Menyembunyikan log TensorFlow
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tqdm import tqdm
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Metric
import tensorflow as tf
from tensorflow.keras.models import load_model
import keras
from tensorflow.keras import backend as K


2025-12-09 06:41:19.230663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765262479.252445     231 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765262479.259088     231 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


**ATTENTION UNET MODEL**

In [2]:
class F1Score(tf.keras.metrics.Metric):
    """Custom F1 Score metric"""
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)
    
    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + K.epsilon()))
    
    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

def attention_gate(gate_signal, skip_connection, inter_channels):
    """
    Attention Gate untuk memfokuskan pada region yang relevan
    
    Args:
        gate_signal: Input dari decoder (gating signal)
        skip_connection: Input dari encoder (skip connection)
        inter_channels: Jumlah channel intermediate
    """
    # Theta pathway - dari skip connection
    theta_x = layers.Conv2D(inter_channels, (1, 1), strides=(1, 1), padding='same', use_bias=False)(skip_connection)
    
    # Phi pathway - dari gating signal
    phi_g = layers.Conv2D(inter_channels, (1, 1), strides=(1, 1), padding='same', use_bias=False)(gate_signal)
    
    # Upsample phi_g jika ukurannya berbeda dengan theta_x
    phi_g_upsampled = layers.UpSampling2D(size=(skip_connection.shape[1] // gate_signal.shape[1], 
                                               skip_connection.shape[2] // gate_signal.shape[2]))(phi_g)
    
    # Add dan apply ReLU
    concat_xg = layers.add([theta_x, phi_g_upsampled])
    act_xg = layers.Activation('relu')(concat_xg)
    
    # Psi pathway - menghasilkan attention coefficients
    psi = layers.Conv2D(1, (1, 1), padding='same', use_bias=True)(act_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)
    
    # Upsample attention coefficients untuk matching dimensi
    upsample_psi = layers.UpSampling2D(size=(1, 1))(sigmoid_xg)
    
    # Apply attention coefficients ke skip connection
    y = layers.multiply([skip_connection, upsample_psi])
    
    # Optional: output gate untuk regularisasi
    result = layers.Conv2D(skip_connection.shape[-1], (1, 1), padding='same')(y)
    result = layers.BatchNormalization()(result)
    
    return result

def attention_unet_model(input_size=(256, 256, 1)):
    """
    Attention U-Net model untuk deteksi titik api
    Dikembangkan dari model U-Net yang sudah ada dengan penambahan attention gates
    """
    inputs = tf.keras.Input(input_size)
    
    # Encoder (sama seperti U-Net original dengan sedikit modifikasi)
    # Block 1
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
    c1 = layers.BatchNormalization()(c1)  # Tambah BatchNorm untuk stabilitas
    p1 = layers.MaxPooling2D((2, 2))(c1)
    p1 = layers.Dropout(0.1)(p1)
    
    # Block 2
    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
    c2 = layers.BatchNormalization()(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    p2 = layers.Dropout(0.1)(p2)
    
    # Block 3
    c3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c3)
    c3 = layers.BatchNormalization()(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)
    p3 = layers.Dropout(0.2)(p3)
    
    # Bottleneck
    bn = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p3)
    bn = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(bn)
    bn = layers.BatchNormalization()(bn)
    bn = layers.Dropout(0.3)(bn)
    
    # Decoder with Attention Gates
    # Upsampling Block 1 dengan Attention Gate
    u1 = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(bn)
    # Attention gate antara u1 (gating) dan c3 (skip connection)
    att1 = attention_gate(gate_signal=u1, skip_connection=c3, inter_channels=32)
    u1 = layers.concatenate([u1, att1])  # Gunakan attended features
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c4)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Dropout(0.2)(c4)
    
    # Upsampling Block 2 dengan Attention Gate  
    u2 = layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(c4)
    # Attention gate antara u2 (gating) dan c2 (skip connection)
    att2 = attention_gate(gate_signal=u2, skip_connection=c2, inter_channels=16)
    u2 = layers.concatenate([u2, att2])  # Gunakan attended features
    c5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
    c5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c5)
    c5 = layers.BatchNormalization()(c5)
    c5 = layers.Dropout(0.1)(c5)
    
    # Upsampling Block 3 dengan Attention Gate
    u3 = layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same')(c5)
    # Attention gate antara u3 (gating) dan c1 (skip connection)
    att3 = attention_gate(gate_signal=u3, skip_connection=c1, inter_channels=8)
    u3 = layers.concatenate([u3, att3])  # Gunakan attended features
    c6 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(u3)
    c6 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c6)
    c6 = layers.BatchNormalization()(c6)
    c6 = layers.Dropout(0.1)(c6)
    
    # Output layer
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c6)
    
    model = tf.keras.Model(inputs, outputs, name='Attention_UNet')
    return model

**UNET MODEL**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, backend as K

# ======= METRIK KUSTOM (tetap) =======
@tf.keras.utils.register_keras_serializable()
class F1Score(tf.keras.metrics.Metric):
    """Custom F1 Score metric"""
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)
    
    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + K.epsilon()))
    
    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

# ======= U-NET TANPA ATTENTION =======
def unet_model(input_size=(256, 256, 1)):
    """
    Plain U-Net (tanpa attention gate).
    Encoder/decoder mengikuti konfigurasi model sebelumnya (16-32-64-128).
    """
    inputs = tf.keras.Input(input_size)

    # ----- Encoder -----
    # Block 1
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
    c1 = layers.BatchNormalization()(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    p1 = layers.Dropout(0.1)(p1)

    # Block 2
    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
    c2 = layers.BatchNormalization()(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    p2 = layers.Dropout(0.1)(p2)

    # Block 3
    c3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c3)
    c3 = layers.BatchNormalization()(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)
    p3 = layers.Dropout(0.2)(p3)

    # ----- Bottleneck -----
    bn = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p3)
    bn = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(bn)
    bn = layers.BatchNormalization()(bn)
    bn = layers.Dropout(0.3)(bn)

    # ----- Decoder (tanpa attention, langsung concat) -----
    # Up 1
    u1 = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(bn)
    u1 = layers.Concatenate()([u1, c3])
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c4)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Dropout(0.2)(c4)

    # Up 2
    u2 = layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(c4)
    u2 = layers.Concatenate()([u2, c2])
    c5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
    c5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c5)
    c5 = layers.BatchNormalization()(c5)
    c5 = layers.Dropout(0.1)(c5)

    # Up 3
    u3 = layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same')(c5)
    u3 = layers.Concatenate()([u3, c1])
    c6 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(u3)
    c6 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c6)
    c6 = layers.BatchNormalization()(c6)
    c6 = layers.Dropout(0.1)(c6)

    # Output
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c6)

    model = tf.keras.Model(inputs, outputs, name='UNet')
    return model

# ---- contoh pemakaian (opsional) ----
# model = unet_model(input_size=(256, 256, 1))
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=[F1Score(name='f1'), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
# model.summary()


# LOAD DATA

In [3]:
import os
import numpy as np
import tensorflow as tf

# =========================
# Lokasi berkas (sesuaikan)
# =========================
PATH_TRAIN = "/kaggle/input/datadeteksikebakaran/datasetIR(7)_train.npz"
PATH_VAL   = "/kaggle/input/datadeteksikebakaran/datasetIR(7)_val.npz"

# =========================
# Util memuat 1 split
# =========================
def load_split(npz_path):
    """
    Memuat split dari .npz berisi:
      - input_np: (N, H, W, C)
      - target_np: (N, H, W) atau (N, H, W, K)
      - file_names: (N,)
    Mengembalikan (X, y, names) sebagai numpy arrays.
    """
    if not os.path.isfile(npz_path):
        raise FileNotFoundError(f"File tidak ditemukan: {npz_path}")

    with np.load(npz_path, allow_pickle=True) as d:
        X = d["input_np"]
        y = d["target_np"]
        names = d["file_names"]

    # Pastikan tipe float32 untuk kompatibilitas model
    if not isinstance(X, np.ndarray):
        X = np.array(X)
    if not isinstance(y, np.ndarray):
        y = np.array(y)
    X = X.astype(np.float32)
    y = y.astype(np.float32)

    # Jaga bentuk channel-last (kalau input 3D -> tambahkan channel tunggal)
    if X.ndim == 3:
        X = X[..., None]  # (N, H, W) -> (N, H, W, 1)

    # Jika target perlu channel dim (tergantung arsitektur model), contoh:
    # if y.ndim == 3 and NEEDS_CHANNEL:
    #     y = y[..., None]

    # Konversi file_names ke list of str
    try:
        names = names.tolist()
    except Exception:
        names = list(names)
    names = [str(n) for n in names]

    return X, y, names

# =========================
# Muat train & val
# =========================
X_train, y_train, names_train = load_split(PATH_TRAIN)
X_val,   y_val,   names_val   = load_split(PATH_VAL)

print("Train:", X_train.shape, y_train.shape, "N files:", len(names_train))
print("Val  :", X_val.shape,   y_val.shape,   "N files:", len(names_val))

# =========================
# tf.data pipeline
# =========================
AUTOTUNE   = tf.data.AUTOTUNE
BATCH_SIZE = 8  # sesuaikan dengan GPU/memori

def make_dataset(X, y, batch_size=BATCH_SIZE, training=True):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if training:
        ds = ds.shuffle(buffer_size=len(X), reshuffle_each_iteration=True)
    ds = ds.batch(batch_size, drop_remainder=False)
    ds = ds.prefetch(AUTOTUNE)
    return ds

ds_train = make_dataset(X_train, y_train, training=True)
ds_val   = make_dataset(X_val,   y_val,   training=False)

print("Contoh nama file train[0]:", names_train[0] if len(names_train) else "(kosong)")
print("Contoh nama file val[0]  :", names_val[0]   if len(names_val)   else "(kosong)")


Train: (369, 256, 256, 1) (369, 256, 256) N files: 369
Val  : (80, 256, 256, 1) (80, 256, 256) N files: 80


I0000 00:00:1765262491.139221     231 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1765262491.139827     231 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Contoh nama file train[0]: 20190801_0230
Contoh nama file val[0]  : 20190803_1440


In [4]:
print("Train X:", X_train.shape, "| dtype:", X_train.dtype)
print("Train y:", y_train.shape, "| dtype:", y_train.dtype)
print("Val   X:", X_val.shape,   "| dtype:", X_val.dtype)
print("Val   y:", y_val.shape,   "| dtype:", y_val.dtype)

print("N train files:", len(names_train))
print("N val files  :", len(names_val))


Train X: (369, 256, 256, 1) | dtype: float32
Train y: (369, 256, 256) | dtype: float32
Val   X: (80, 256, 256, 1) | dtype: float32
Val   y: (80, 256, 256) | dtype: float32
N train files: 369
N val files  : 80


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, Callback

# =========================
# Opsi: selaraskan bentuk data
# =========================
# Pastikan tipe & shape sesuai (model biasanya output (N,H,W,1) dengan sigmoid)
X_train = X_train.astype(np.float32)
X_val   = X_val.astype(np.float32)
y_train = y_train.astype(np.float32)
y_val   = y_val.astype(np.float32)

# Jika target masih (N,H,W) dan model mengharapkan (N,H,W,1), aktifkan ini:
if y_train.ndim == 3:
    y_train = y_train[..., None]
if y_val.ndim == 3:
    y_val = y_val[..., None]

# Jika input masih (N,H,W) dan model mengharapkan (N,H,W,1), aktifkan ini juga:
if X_train.ndim == 3:
    X_train = X_train[..., None]
if X_val.ndim == 3:
    X_val = X_val[..., None]

# =========================
# Metrik F1 kustom (threshold = 0.5 default)
# =========================
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        # y_pred diasumsikan probabilitas; Precision/Recall akan threshold 0.5
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2.0 * ((precision * recall) / (precision + recall + K.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# =========================
# Definisi model
# =========================
# Asumsikan kamu punya fungsi pembuat model: unet_model() atau attention_unet_model()
model = attention_unet_model()   # atau: model = attention_unet_model()
# model = unet_model()
# Pastikan layer terakhir sigmoid dan shape output == y_train.shape[1:]

# =========================
# Loss functions
# =========================
def focal_loss(alpha=0.25, gamma=2.0):
    def focal_loss_fixed(y_true, y_pred):
        # y_pred: probabilitas (0..1)
        y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1.0 - K.epsilon())
        pt = tf.where(tf.equal(y_true, 1.0), y_pred, 1.0 - y_pred)
        alpha_t = tf.where(tf.equal(y_true, 1.0), alpha, 1.0 - alpha)
        loss = -alpha_t * K.pow(1.0 - pt, gamma) * K.log(pt)
        return tf.reduce_mean(loss)
    return focal_loss_fixed

def weighted_binary_crossentropy(pos_weight=15.0):
    """
    pos_weight > 1.0 akan memberi penalti lebih besar untuk kelas positif.
    Bekerja di ruang probabilitas (bukan logits).
    """
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1.0 - K.epsilon())
        loss_pos = -pos_weight * y_true * K.log(y_pred)
        loss_neg = -(1.0 - y_true) * K.log(1.0 - y_pred)
        loss_all = loss_pos + loss_neg
        return tf.reduce_mean(loss_all)
    return loss

# =========================
# Optimizer & Compile
# =========================
learning_rate = 1e-4
optimizer = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

model.compile(
    optimizer=optimizer,
    # pilih salah satu:
    loss=weighted_binary_crossentropy(pos_weight=15.0),
    # loss=focal_loss(alpha=0.25, gamma=2.0),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='acc'),
        tf.keras.metrics.AUC(name='auc_roc', curve='ROC'),
        tf.keras.metrics.AUC(name='auc_pr',  curve='PR'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score(name='f1')
    ]
)

# =========================
# Callbacks
# =========================
class CustomModelCheckpoint(Callback):
    def __init__(self, filepath, save_freq=100):
        super(CustomModelCheckpoint, self).__init__()
        self.filepath = filepath
        self.save_freq = save_freq
        
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            filename = self.filepath.format(epoch=epoch + 1)
            self.model.save(filename)
            print(f"\nModel disimpan di {filename} (Epoch {epoch + 1})")

class CustomHistory(Callback):
    def __init__(self, save_interval=100, save_dir='history'):
        super(CustomHistory, self).__init__()
        self.save_interval = save_interval
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        self.history = {}
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        for key, value in logs.items():
            self.history.setdefault(key, []).append(value)
        if (epoch + 1) % self.save_interval == 0:
            save_path = os.path.join(self.save_dir, f'history_epoch_{epoch + 1}.npy')
            np.save(save_path, self.history, allow_pickle=True)
            print(f'History disimpan di {save_path}')

# Direktori kerja & checkpoint
base_dir = '/kaggle/working/deteksi_kebakaran'
model_dir = os.path.join(base_dir, 'saved_models')
history_dir = os.path.join(base_dir, 'training_history')
os.makedirs(model_dir, exist_ok=True)
os.makedirs(history_dir, exist_ok=True)

checkpoint_filepath = os.path.join(model_dir, 'model_epoch_{epoch:04d}.keras')

model_checkpoint_callback = CustomModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq=100
)

custom_history_callback = CustomHistory(
    save_interval=100,
    save_dir=history_dir
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    min_lr=1e-7,
    verbose=1
)

# =========================
# Training
# =========================
BATCH_SIZE = 8  # sesuaikan
EPOCHS = 200

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[model_checkpoint_callback, reduce_lr, custom_history_callback],
    verbose=1
)


# testing the test data

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tqdm import trange

# ======== PARAMETER ========
# Dataset paths
PATH_TEST = "/kaggle/input/datadeteksikebakaran/datasetIR(7)_test.npz"
# (Opsional) gunakan validation untuk cari threshold optimal
PATH_VAL  = "/kaggle/input/datadeteksikebakaran/datasetIR(7)_val.npz"

# (Opsional) kalau model TIDAK ada di memori:
MODEL_PATH = "/kaggle/input/datadeteksikebakaran/deteksi_kebakaran_2sensorNasa_AttentionUnet_buffer10m_noFilter/saved_models/model_epoch_0200.keras"
# MODEL_PATH = None  # set jadi path file .keras kalau mau load dari disk

BATCH_SIZE = 8

# ======== UTIL ========
def load_split(npz_path):
    with np.load(npz_path, allow_pickle=True) as d:
        X = d["input_np"].astype(np.float32)
        y = d["target_np"].astype(np.float32)
        names = d["file_names"].tolist()
    # Jaga channel last
    if X.ndim == 3: X = X[..., None]
    if y.ndim == 3: y = y[..., None]
    return X, y, names

def make_dataset(X, y, batch=BATCH_SIZE, training=False):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if training:
        ds = ds.shuffle(len(X), reshuffle_each_iteration=True)
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

# ======== MUAT DATA TEST (dan VAL opsional) ========
X_test, y_test, names_test = load_split(PATH_TEST)
ds_test = make_dataset(X_test, y_test, training=False)
print("Test:", X_test.shape, y_test.shape, "N files:", len(names_test))

# (Opsional) untuk threshold optimal
X_val_opt, y_val_opt = None, None
if PATH_VAL and os.path.isfile(PATH_VAL):
    X_val_opt, y_val_opt, _ = load_split(PATH_VAL)
    ds_val_opt = make_dataset(X_val_opt, y_val_opt, training=False)
    print("Val (for threshold opt):", X_val_opt.shape, y_val_opt.shape)

# ======== MUAT / PAKAI MODEL ========
if (MODEL_PATH is not None) and os.path.isfile(MODEL_PATH):
    # Tidak perlu compile untuk prediksi/evaluasi manual
    model = load_model(MODEL_PATH, compile=False)   # <— kuncinya di sini
    print("Model loaded (compile=False) from:", MODEL_PATH)
else:
    assert 'model' in globals(), "Model tidak ditemukan di memori dan MODEL_PATH tidak diset."
    print("Using in-memory model.")


# ======== PREDIKSI PROBABILITAS ========
y_prob_test = model.predict(ds_test, verbose=1)
# pastikan shape sama (N,H,W,1)
if y_prob_test.ndim == 3: y_prob_test = y_prob_test[..., None]

# ======== METRIK BERBASIS AUC (tanpa threshold) ========
# Flatten ke 1D untuk per-pixel metrics
y_true_flat = y_test.reshape(-1).astype(np.int32)
y_prob_flat = y_prob_test.reshape(-1).astype(np.float32)

from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, roc_curve

roc_auc = roc_auc_score(y_true_flat, y_prob_flat)
pr_auc  = average_precision_score(y_true_flat, y_prob_flat)  # = PR-AUC

print(f"ROC-AUC (test): {roc_auc:.4f}")
print(f"PR-AUC  (test): {pr_auc:.4f}")

# ======== BOOTSTRAP 95% CI UNTUK ROC-AUC DAN PR-AUC ========
from numpy.random import default_rng

# Jumlah timestamp di test set (unit bootstrap)
N_ts = y_test.shape[0]
print("N test timestamps:", N_ts)

# Jumlah bootstrap replicates (boleh 200–1000; 500 kompromi bagus)
B = 200
rng = default_rng(42)

roc_aucs_bs = []
pr_aucs_bs  = []

for b in trange(B, desc="Bootstrapping ROC/PR", ncols=80):
    idx = rng.integers(0, N_ts, size=N_ts)

    y_true_b = y_test[idx].reshape(-1).astype(np.int32)
    y_prob_b = y_prob_test[idx].reshape(-1).astype(np.float32)

    if len(np.unique(y_true_b)) < 2:
        continue

    roc_b = roc_auc_score(y_true_b, y_prob_b)
    pr_b  = average_precision_score(y_true_b, y_prob_b)

    roc_aucs_bs.append(roc_b)
    pr_aucs_bs.append(pr_b)

roc_aucs_bs = np.array(roc_aucs_bs)
pr_aucs_bs  = np.array(pr_aucs_bs)

print(f"Effective bootstrap replicates used: {len(roc_aucs_bs)}")

# 4) Ambil percentile 2.5% dan 97.5% → CI 95%
roc_ci = np.percentile(roc_aucs_bs, [2.5, 97.5])
pr_ci  = np.percentile(pr_aucs_bs,  [2.5, 97.5])

print(f"ROC-AUC  (test): {roc_auc:.4f}  |  95% CI ≈ [{roc_ci[0]:.4f}, {roc_ci[1]:.4f}]")
print(f"PR-AUC   (test): {pr_auc:.4f}   |  95% CI ≈ [{pr_ci[0]:.4f}, {pr_ci[1]:.4f}]")


# ======== THRESHOLD: 0.5 dan (opsional) optimal dari validation ========
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

def binarize(prob, thr):
    return (prob >= thr).astype(np.uint8)

def metrics_at_threshold(y_true, y_prob, thr):
    y_pred = binarize(y_prob, thr)
    P = precision_score(y_true, y_pred, zero_division=0)
    R = recall_score(y_true, y_pred, zero_division=0)
    F1 = f1_score(y_true, y_pred, zero_division=0)
    ACC = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    return {"threshold": thr, "precision": P, "recall": R, "f1": F1, "accuracy": ACC, "cm": cm}

# 1) Evaluasi di threshold 0.5
m05 = metrics_at_threshold(y_true_flat, y_prob_flat, 0.5)
print("\n== Metrics @ threshold=0.5 (test) ==")
print(f"Precision: {m05['precision']:.4f}  Recall: {m05['recall']:.4f}  F1: {m05['f1']:.4f}  Acc: {m05['accuracy']:.4f}")
print("Confusion matrix:\n", m05['cm'])

# 2) (Opsional) Cari threshold optimal di validation (maks F1), lalu terapkan ke test
thr_opt = None
if X_val_opt is not None:
    y_prob_val = model.predict(ds_val_opt, verbose=0)
    if y_prob_val.ndim == 3: y_prob_val = y_prob_val[..., None]
    y_true_val_flat = y_val_opt.reshape(-1).astype(np.int32)
    y_prob_val_flat = y_prob_val.reshape(-1).astype(np.float32)

    precisions, recalls, thresholds = precision_recall_curve(y_true_val_flat, y_prob_val_flat)
    # thresholds array panjangnya len(precisions)-1; hitung F1 untuk setiap threshold
    eps = 1e-9
    f1s = 2 * precisions[:-1] * recalls[:-1] / (precisions[:-1] + recalls[:-1] + eps)
    best_idx = np.argmax(f1s)
    thr_opt = thresholds[best_idx]
    print(f"\n[VAL] Best F1 threshold ≈ {thr_opt:.4f} | P={precisions[best_idx]:.4f} R={recalls[best_idx]:.4f} F1={f1s[best_idx]:.4f}")

    # Terapkan ke TEST
    mOPT = metrics_at_threshold(y_true_flat, y_prob_flat, thr_opt)
    print("\n== Metrics @ threshold=ValBestF1 (applied to test) ==")
    print(f"Threshold: {mOPT['threshold']:.4f}")
    print(f"Precision: {mOPT['precision']:.4f}  Recall: {mOPT['recall']:.4f}  F1: {mOPT['f1']:.4f}  Acc: {mOPT['accuracy']:.4f}")
    print("Confusion matrix:\n", mOPT['cm'])

# ======== OPSIONAL: SIMPAN SKOR PREDIKSI UNTUK ANALISIS LANJUT ========
# Misal ingin disimpan agar tidak perlu predict ulang untuk plot PR/ROC nanti:
np.save("/kaggle/working/y_prob_test.npy", y_prob_test)
np.save("/kaggle/working/y_true_test.npy", y_test)


Test: (75, 256, 256, 1) (75, 256, 256, 1) N files: 75
Val (for threshold opt): (80, 256, 256, 1) (80, 256, 256, 1)
Model loaded (compile=False) from: /kaggle/input/datadeteksikebakaran/deteksi_kebakaran_2sensorNasa_AttentionUnet_buffer10m_noFilter/saved_models/model_epoch_0200.keras


I0000 00:00:1765262507.624815     266 service.cc:148] XLA service 0x7fb97c0037a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765262507.624865     266 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1765262507.624870     266 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1765262507.826086     266 cuda_dnn.cc:529] Loaded cuDNN version 90300


 7/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

I0000 00:00:1765262511.229763     266 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 442ms/step
ROC-AUC (test): 0.9223
PR-AUC  (test): 0.3601
N test timestamps: 75


Bootstrapping ROC/PR: 100%|███████████████████| 200/200 [13:31<00:00,  4.06s/it]


Effective bootstrap replicates used: 200
ROC-AUC  (test): 0.9223  |  95% CI ≈ [0.9142, 0.9309]
PR-AUC   (test): 0.3601   |  95% CI ≈ [0.3055, 0.4139]

== Metrics @ threshold=0.5 (test) ==
Precision: 0.2255  Recall: 0.6075  F1: 0.3289  Acc: 0.9620
Confusion matrix:
 [[4682696  157180]
 [  29567   45757]]

[VAL] Best F1 threshold ≈ 0.8056 | P=0.3871 R=0.3911 F1=0.3891

== Metrics @ threshold=ValBestF1 (applied to test) ==
Threshold: 0.8056
Precision: 0.4243  Recall: 0.3947  F1: 0.4090  Acc: 0.9825
Confusion matrix:
 [[4799541   40335]
 [  45594   29730]]


# Find the optimal threshold based on the objective (precision-first / recall-first / F1-score

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

def sweep_thresholds(y_true_flat, y_prob_flat, num=200):
    thr_grid = np.linspace(0.0, 1.0, num=num, endpoint=True)
    rows = []
    for thr in thr_grid:
        y_pred = (y_prob_flat >= thr).astype(np.uint8)
        P = precision_score(y_true_flat, y_pred, zero_division=0)
        R = recall_score(y_true_flat, y_pred, zero_division=0)
        F1 = 2*P*R/(P+R+1e-9)
        ACC = accuracy_score(y_true_flat, y_pred)
        rows.append((thr, P, R, F1, ACC))
    return np.array(rows)  # cols: thr, P, R, F1, ACC

grid = sweep_thresholds(y_true_flat, y_prob_flat, num=401)

# contoh pemilihan:
# a) threshold dengan F1 maksimum
thr_f1 = grid[np.argmax(grid[:,3]), 0]

# b) threshold minimal precision (mis. ≥ 0.60) dengan recall tertinggi
mask = grid[:,1] >= 0.60
thr_p60 = grid[mask][np.argmax(grid[mask][:,2]), 0] if np.any(mask) else None

# c) threshold minimal recall (mis. ≥ 0.60) dengan precision tertinggi
mask = grid[:,2] >= 0.60
thr_r60 = grid[mask][np.argmax(grid[mask][:,1]), 0] if np.any(mask) else None

print("Best F1 thr:", thr_f1)
print("Precision≥0.60 thr:", thr_p60)
print("Recall≥0.60 thr:", thr_r60)

def report_at_thr(y_true, y_prob, thr, title):
    y_pred = (y_prob >= thr).astype(np.uint8)
    P = precision_score(y_true, y_pred, zero_division=0)
    R = recall_score(y_true, y_pred, zero_division=0)
    F1 = f1_score(y_true, y_pred, zero_division=0)
    ACC = accuracy_score(y_true, y_pred)
    CM = confusion_matrix(y_true, y_pred)
    print(f"\n== {title} ==")
    print(f"thr={thr:.4f} | P={P:.4f} R={R:.4f} F1={F1:.4f} ACC={ACC:.4f}")
    print("Confusion matrix:\n", CM)

report_at_thr(y_true_flat, y_prob_flat, thr_f1, "Test @ Best F1(thr grid)")
if thr_p60 is not None: report_at_thr(y_true_flat, y_prob_flat, thr_p60, "Test @ Precision≥0.60")
if thr_r60 is not None: report_at_thr(y_true_flat, y_prob_flat, thr_r60, "Test @ Recall≥0.60")


In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import ListedColormap
from shapely.geometry import box
import scipy.ndimage as ndi
from scipy.spatial import cKDTree
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, confusion_matrix
)

# ========= PATHS =========
PATH_TEST_NPZ = "/kaggle/input/datadeteksikebakaran/datasetIR(7)_test.npz"
MODEL_PATH = "/kaggle/input/datadeteksikebakaran/deteksi_kebakaran_2sensorNasa_AttentionUnet/saved_models/model_epoch_0200.keras"

# ========= OUTPUT (SAVE FIGS) =========
SAVE_DIR   = "/kaggle/working/deteksi_kebakaran"  # <-- folder simpan gambar
SAVE_FIGS  = True                                 # simpan gambar?
SHOW_FIGS  = False                                # tampilkan jendela gambar?
SAVE_DPI   = 300                                  # resolusi simpan

# ========= PARAMS =========
MIN_SIZE = 9
THR = 0.8056
MATCH_RADIUS = 8.0
N_SAMPLES_TO_PLOT = 75

# --- Opsi pemilihan & pengurutan plot ---
SELECT_TIMESTAMPS = ['20190907_1510','20191029_1450','20190907_0250','20190907_0530']  # contoh: ['20190907_0620','20191006_0720','20190803_1220']
PLOT_TOP_K = N_SAMPLES_TO_PLOT

# ========= CUSTOM OBJECTS =========
@tf.keras.utils.register_keras_serializable()
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()
    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)
    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2.0 * ((p * r) / (p + r + tf.keras.backend.epsilon()))
    def reset_states(self):
        super().reset_states()
        self.precision.reset_states()
        self.recall.reset_states()

@tf.keras.utils.register_keras_serializable()
def weighted_binary_crossentropy(pos_weight=15.0):
    def loss(y_true, y_pred):
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        weights = y_true * pos_weight + (1 - y_true) * 1.0
        return tf.reduce_mean(weights * bce)
    return loss

def remove_small_blobs(mask, min_size=3):
    lab, n = ndi.label(mask)
    if n == 0:
        return mask
    counts = np.bincount(lab.ravel())
    keep = np.isin(lab, np.where(counts >= min_size)[0])
    return (keep & (lab > 0)).astype(np.uint8)

MIN_SIZE_OBJ = max(3, MIN_SIZE)

def label_and_centroids(mask, min_size=MIN_SIZE_OBJ):
    structure = np.ones((3,3), dtype=np.uint8)  # 8-connectivity
    lab, n = ndi.label(mask.astype(np.uint8), structure=structure)
    if n == 0:
        return np.zeros_like(mask, dtype=np.int32), np.empty((0,2), dtype=float)
    sizes = np.bincount(lab.ravel())
    keep = np.isin(lab, np.where(sizes >= min_size)[0])
    lab = lab * keep
    lab, _ = ndi.label(lab > 0, structure=structure)
    if lab.max() == 0:
        return lab.astype(np.int32), np.empty((0,2), dtype=float)
    cents = ndi.center_of_mass(np.ones_like(mask, dtype=np.uint8), labels=lab, index=range(1, lab.max()+1))
    return lab.astype(np.int32), np.array(cents, dtype=float)  # (K,2) -> (y,x)

def greedy_match_by_radius(cents_gt, cents_pred, radius=MATCH_RADIUS):
    if len(cents_gt) == 0 and len(cents_pred) == 0:
        return [], [], []
    if len(cents_gt) == 0:
        return [], [], list(range(len(cents_pred)))
    if len(cents_pred) == 0:
        return [], list(range(len(cents_gt))), []
    tree = cKDTree(cents_pred)
    dists, idxs = tree.query(cents_gt, k=1, distance_upper_bound=radius)
    matched, used_pred = [], set()
    for i_gt, (d, j_pred) in enumerate(zip(dists, idxs)):
        if np.isfinite(d) and j_pred < len(cents_pred) and j_pred not in used_pred:
            matched.append((i_gt, j_pred)); used_pred.add(j_pred)
    matched_gt = {i for i,_ in matched}
    matched_pr = {j for _,j in matched}
    un_gt = [i for i in range(len(cents_gt))  if i not in matched_gt]
    un_pr = [j for j in range(len(cents_pred)) if j not in matched_pr]
    return matched, un_gt, un_pr

def object_level_counts(gt_mask, pred_mask, radius=MATCH_RADIUS, min_size=MIN_SIZE_OBJ):
    _, cg = label_and_centroids(gt_mask,  min_size=min_size)
    _, cp = label_and_centroids(pred_mask, min_size=min_size)
    matched, un_gt, un_pr = greedy_match_by_radius(cg, cp, radius=radius)
    TP_obj = len(matched); FN_obj = len(un_gt); FP_obj = len(un_pr)
    return TP_obj, FP_obj, FN_obj

# ========= LOAD MODEL & DATA =========
model = tf.keras.models.load_model(
    MODEL_PATH,
    custom_objects={
        'F1Score': F1Score,
        'weighted_binary_crossentropy': weighted_binary_crossentropy,
        'loss': weighted_binary_crossentropy()
    }
)
print("Loaded model from:", MODEL_PATH)

with np.load(PATH_TEST_NPZ, allow_pickle=True) as d:
    X_test = d["input_np"].astype(np.float32)
    y_test = d["target_np"].astype(np.float32)
    names_test = d["file_names"].tolist()

if X_test.ndim == 3: X_test = X_test[..., None]
if y_test.ndim == 3: y_test = y_test[..., None]

print("Test shapes:", X_test.shape, y_test.shape, "| N files:", len(names_test))

# ========= GEO =========
shapefile_path = '/kaggle/input/datadeteksikebakaran/gadm41_IDN_1.shp'
boundaries = gpd.read_file(shapefile_path)

lat_lower = -3.650602881343665
lat_upper =  1.469398964128776
lon_lower = 110.73089588122669
lon_upper = 115.85088234776212
extent = [lon_lower, lon_upper, lat_lower, lat_upper]

roi = gpd.GeoDataFrame(geometry=[box(lon_lower, lat_lower, lon_upper, lat_upper)], crs='EPSG:4326')
provinsi_kalteng_roi = gpd.overlay(boundaries, roi, how='intersection')

binary_cmap = ListedColormap([[0, 0, 0, 0.0], [1, 0, 0, 0.85]])

# ========= PLOTTING (4 panel) =========
def plot_sample_with_boundary(input_image, pred_mask, gt_mask, boundaries, file_name,
                              pc, oe, ce, f1_obj, flip=True, scatter_max=5000,
                              save_dir=None, save_basename=None, save_dpi=300, show=False):
    H, W = input_image.shape[:2]
    img2d = input_image.squeeze()

    lon = np.linspace(lon_lower, lon_upper, W)
    lat = np.linspace(lat_lower, lat_upper, H)
    lon_mesh, lat_mesh = np.meshgrid(lon, lat)

    fig, axs = plt.subplots(1, 4, figsize=(24, 5), subplot_kw={'projection': ccrs.PlateCarree()})

    # Panel 1: Input
    base = np.flipud(img2d) if flip else img2d
    im0 = axs[0].pcolormesh(lon_mesh, lat_mesh, base, cmap='RdYlBu_r', transform=ccrs.PlateCarree())
    boundaries.boundary.plot(ax=axs[0], edgecolor='black')
    axs[0].set_title(f'Input Image: {file_name}')
    axs[0].set_extent(extent, crs=ccrs.PlateCarree())
    axs[0].coastlines(resolution='10m', color='black', linewidth=1)
    axs[0].add_feature(cfeature.BORDERS, linestyle=':', linewidth=1, edgecolor='black')
    fig.colorbar(im0, ax=axs[0], orientation='vertical', fraction=0.046, pad=0.04)

    def idx_to_lonlat_southup(mask):
        yy, xx = np.where(mask > 0)
        if len(yy) == 0:
            return np.array([]), np.array([])
        Hh, Ww = mask.shape
        lon_arr = lon_lower + (xx + 0.5) * (lon_upper - lon_lower) / Ww
        lat_arr = lat_lower + (yy + 0.5) * (lat_upper - lat_lower) / Hh
        if len(lon_arr) > scatter_max:
            sel = np.random.choice(len(lon_arr), scatter_max, replace=False)
            lon_arr, lat_arr = lon_arr[sel], lat_arr[sel]
        return lon_arr, lat_arr

    # Panel 2: Prediction
    pred_show = np.flipud(pred_mask) if flip else pred_mask
    if np.count_nonzero(pred_show) > 0:
        axs[1].imshow(pred_show, cmap=binary_cmap, extent=extent, origin='lower',
                      transform=ccrs.PlateCarree(), alpha=0.9, interpolation='nearest', zorder=2)
        axs[1].contour(lon_mesh, lat_mesh, pred_show, levels=[0.5], colors='red',
                       linewidths=0.6, transform=ccrs.PlateCarree(), zorder=3)
        lon_p, lat_p = idx_to_lonlat_southup(pred_show)
        if lon_p.size:
            axs[1].scatter(lon_p, lat_p, s=6, c='red', transform=ccrs.PlateCarree(),
                           edgecolors='none', linewidths=0, zorder=4, label='Predicted Fire')
    boundaries.boundary.plot(ax=axs[1], edgecolor='black', zorder=5)
    axs[1].set_title('Predicted Mask')
    axs[1].set_extent(extent, crs=ccrs.PlateCarree())
    axs[1].coastlines(resolution='10m', color='black', linewidth=1)
    axs[1].add_feature(cfeature.BORDERS, linestyle=':', linewidth=1, edgecolor='black')
    axs[1].text(0.5, -0.32,
                f'PC: {pc:.2f}%\nOE: {oe:.2f}%\nCE: {ce:.2f}%\nF1(obj): {f1_obj:.2f}%',
                ha='center', va='center', transform=axs[1].transAxes, fontsize=10,
                bbox=dict(facecolor='white', alpha=0.9))
    axs[1].legend(loc='upper left')

    # Panel 3: Ground Truth
    gt_show = np.flipud(gt_mask) if flip else gt_mask
    if np.count_nonzero(gt_show) > 0:
        axs[2].imshow(gt_show, cmap=binary_cmap, extent=extent, origin='lower',
                      transform=ccrs.PlateCarree(), alpha=0.9, interpolation='nearest', zorder=2)
        axs[2].contour(lon_mesh, lat_mesh, gt_show, levels=[0.5], colors='red',
                       linewidths=0.6, transform=ccrs.PlateCarree(), zorder=3)
        lon_g, lat_g = idx_to_lonlat_southup(gt_show)
        if lon_g.size:
            axs[2].scatter(lon_g, lat_g, s=6, c='red', transform=ccrs.PlateCarree(),
                           edgecolors='none', linewidths=0, zorder=4, label='Ground Truth Fire')
    boundaries.boundary.plot(ax=axs[2], edgecolor='black', zorder=5)
    axs[2].set_title('Ground Truth Mask')
    axs[2].set_extent(extent, crs=ccrs.PlateCarree())
    axs[2].coastlines(resolution='10m', color='black', linewidth=1)
    axs[2].add_feature(cfeature.BORDERS, linestyle=':', linewidth=1, edgecolor='black')
    axs[2].legend(loc='upper left')

    # Panel 4: Overlay TP/FP/FN
    overlay = np.zeros_like(pred_show, dtype=np.uint8)
    TP = (pred_show == 1) & (gt_show == 1)
    FP = (pred_show == 1) & (gt_show == 0)
    FN = (pred_show == 0) & (gt_show == 1)
    overlay[TP] = 1  # green
    overlay[FP] = 2  # red
    overlay[FN] = 3  # orange

    overlay_cmap = ListedColormap([
        (0, 0, 0, 0.0),        # none
        (0.0, 0.6, 0.0, 0.85), # TP
        (0.9, 0.0, 0.0, 0.85), # FP
        (1.0, 0.55, 0.0, 0.85) # FN
    ])

    axs[3].imshow(overlay, cmap=overlay_cmap, extent=extent, origin='lower',
                  transform=ccrs.PlateCarree(), interpolation='nearest', zorder=2)
    boundaries.boundary.plot(ax=axs[3], edgecolor='black', zorder=5)
    axs[3].set_title('Overlay: TP/FP/FN')
    axs[3].set_extent(extent, crs=ccrs.PlateCarree())
    axs[3].coastlines(resolution='10m', color='black', linewidth=1)
    axs[3].add_feature(cfeature.BORDERS, linestyle=':', linewidth=1, edgecolor='black')
    legend_handles = [
        mpatches.Patch(color=(0.0, 0.6, 0.0, 0.85), label='TP'),
        mpatches.Patch(color=(0.9, 0.0, 0.0, 0.85), label='FP'),
        mpatches.Patch(color=(1.0, 0.55, 0.0, 0.85), label='FN'),
    ]
    axs[3].legend(handles=legend_handles, loc='upper left')

    plt.tight_layout()

    # ==== SAVE ====
    if save_dir is not None:
        os.makedirs(save_dir, exist_ok=True)
        if not save_basename:
            save_basename = str(file_name)
        out_path = os.path.join(save_dir, f"{save_basename}.png")
        fig.savefig(out_path, dpi=save_dpi, bbox_inches='tight')
        print(f"[SAVED] {out_path}")

    # show/close
    if show:
        plt.show()
    else:
        plt.close(fig)

# ========= PASS 1: hitung metrik per-sample =========
def _key(n):
    return os.path.splitext(os.path.basename(str(n)))[0]

name_keys = [_key(n) for n in names_test]

TP_OBJ_TOT = FP_OBJ_TOT = FN_OBJ_TOT = 0
per_sample = []

for i in range(len(X_test)):
    x_np = X_test[i]
    y_np = y_test[i]

    y_prob = model.predict(x_np[None, ...], verbose=0)
    y_pred = (y_prob[0, ..., 0] >= THR).astype(np.uint8)
    y_pred = remove_small_blobs(y_pred, min_size=MIN_SIZE)
    y_true = (y_np[..., 0] >= 0.5).astype(np.uint8)

    TPo, FPo, FNo = object_level_counts(y_true, y_pred, radius=MATCH_RADIUS, min_size=MIN_SIZE_OBJ)
    TP_OBJ_TOT += TPo; FP_OBJ_TOT += FPo; FN_OBJ_TOT += FNo

    prec_obj = TPo / (TPo + FPo) if (TPo + FPo) > 0 else 0.0
    rec_obj  = TPo / (TPo + FNo) if (TPo + FNo) > 0 else 0.0
    ce_obj = (1.0 - prec_obj) * 100.0
    oe_obj = (1.0 - rec_obj)  * 100.0
    pc_obj = (TPo) / (TPo + FPo + FNo) * 100.0 if (TPo + FPo + FNo) > 0 else 0.0
    f1_obj = (2*prec_obj*rec_obj/(prec_obj+rec_obj) if (prec_obj+rec_obj)>0 else 0.0) * 100.0

    per_sample.append({
        'i': i,
        'name': name_keys[i],
        'TP': TPo, 'FP': FPo, 'FN': FNo,
        'CE': ce_obj, 'OE': oe_obj, 'PC': pc_obj, 'F1': f1_obj
    })

# ===== Ringkasan mikro (objek) =====
den_ce = TP_OBJ_TOT + FP_OBJ_TOT
den_oe = TP_OBJ_TOT + FN_OBJ_TOT
CE_obj_micro = 100.0 * FP_OBJ_TOT / den_ce if den_ce > 0 else 0.0
OE_obj_micro = 100.0 * FN_OBJ_TOT / den_oe if den_oe > 0 else 0.0
prec_obj_micro = TP_OBJ_TOT / (TP_OBJ_TOT + FP_OBJ_TOT) if (TP_OBJ_TOT + FP_OBJ_TOT) > 0 else 0.0
rec_obj_micro  = TP_OBJ_TOT / (TP_OBJ_TOT + FN_OBJ_TOT) if (TP_OBJ_TOT + FN_OBJ_TOT) > 0 else 0.0
F1_obj_micro   = 2*prec_obj_micro*rec_obj_micro/(prec_obj_micro+rec_obj_micro) if (prec_obj_micro+rec_obj_micro)>0 else 0.0
PC_obj_micro   = 100.0 * TP_OBJ_TOT / (TP_OBJ_TOT + FP_OBJ_TOT + FN_OBJ_TOT) if (TP_OBJ_TOT + FP_OBJ_TOT + FN_OBJ_TOT) > 0 else 0.0

print("\n=== SUMMARY (object-level, micro-average) ===")
print(f"TP_obj={TP_OBJ_TOT}  FP_obj={FP_OBJ_TOT}  FN_obj={FN_OBJ_TOT}")
print(f"Commission error (Obj): {CE_obj_micro:.2f}%")
print(f"Omission error (Obj) : {OE_obj_micro:.2f}%")
print(f"F1 (object)         : {F1_obj_micro*100:.2f}%")
print(f"Object correctness  : {PC_obj_micro:.2f}%")

# ========= PILIH & URUTKAN UNTUK DIPLOT =========
if SELECT_TIMESTAMPS:
    selected_set = set(SELECT_TIMESTAMPS)
    found_names = {d['name'] for d in per_sample}
    missing = [ts for ts in SELECT_TIMESTAMPS if ts not in found_names]
    if missing:
        print("[WARNING] Timestamp tidak ditemukan di test split:", missing)
    plot_list = [d for d in per_sample if d['name'] in selected_set]
else:
    plot_list = per_sample[:]

plot_list.sort(key=lambda d: (d['CE'], d['OE']))
if not SELECT_TIMESTAMPS:
    plot_list = plot_list[:PLOT_TOP_K]

# ========= PLOT & SAVE SESUAI URUTAN =========
if SAVE_FIGS:
    os.makedirs(SAVE_DIR, exist_ok=True)

for rank, d in enumerate(plot_list, start=1):
    i = d['i']
    x_np = X_test[i]
    y_np = y_test[i]
    # recompute untuk kepastian visual
    y_prob = model.predict(x_np[None, ...], verbose=0)
    y_pred = (y_prob[0, ..., 0] >= THR).astype(np.uint8)
    y_pred = remove_small_blobs(y_pred, min_size=MIN_SIZE)
    y_true = (y_np[..., 0] >= 0.5).astype(np.uint8)

    save_basename = f"{rank:03d}_{d['name']}_CE{d['CE']:.1f}_OE{d['OE']:.1f}"
    print(f"[PLOT] {save_basename}")

    plot_sample_with_boundary(
        x_np.squeeze(), y_pred, y_true, provinsi_kalteng_roi, d['name'],
        pc=d['PC'], oe=d['OE'], ce=d['CE'], f1_obj=d['F1'],
        flip=True, scatter_max=4000,
        save_dir=(SAVE_DIR if SAVE_FIGS else None),
        save_basename=save_basename,
        save_dpi=SAVE_DPI,
        show=SHOW_FIGS
    )

# ========= PIXEL-LEVEL (opsional) =========
y_prob_test = model.predict(X_test, verbose=1)
y_true_flat = y_test.reshape(-1).astype(np.int32)
y_prob_flat = y_prob_test.reshape(-1).astype(np.float32)

roc_auc = roc_auc_score(y_true_flat, y_prob_flat)
pr_auc  = average_precision_score(y_true_flat, y_prob_flat)
print(f"\nROC-AUC (test): {roc_auc:.4f}")
print(f"PR-AUC  (test): {pr_auc:.4f}")

y_pred_flat = (y_prob_flat >= THR).astype(np.uint8)
P = precision_score(y_true_flat, y_pred_flat, zero_division=0)
R = recall_score(y_true_flat, y_pred_flat, zero_division=0)
F1 = f1_score(y_true_flat, y_pred_flat, zero_division=0)
ACC = accuracy_score(y_true_flat, y_pred_flat)
CM = confusion_matrix(y_true_flat, y_pred_flat)

print(f"\n== Pixel-level metrics @ threshold={THR:.4f} ==")
print(f"Precision: {P:.4f}  Recall: {R:.4f}  F1: {F1:.4f}  Accuracy: {ACC:.4f}")
print("Confusion matrix:\n", CM)
